Fontos! először futtatsd ezt, hogy bemásold Gitről a dolgokat.

<b> MÉGFONTOSABB! </b> Menteni úgy tudod a notebook-ot, hogy Fájl -> Másolat mentése Githubra

In [ ]:
!git clone https://github.com/konderal333/HGT-2022-EmDomArDon.git

Cloning into 'HGT-2022-EmDomArDon'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 24 (delta 10), reused 18 (delta 5), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [ ]:
#Google drive for bigger data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Scraping

In [ ]:
!pip install BeautifulSoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

In [ ]:
# this times ten is the number of articles to scrape 
cikkszam = 1000

# 2000 articles need approx 20 minutes to scrape

In [9]:
page = requests.get("https://telex.hu/legfrissebb")   
soup = BeautifulSoup(page.text, 'html.parser')
soup.prettify

titles = []
headline = []
textbody = []



#for the first page
links = soup.find_all('a',{'class':'list__item__title'})
heads = soup.find_all('p',{'class':'list__item__lead hasHighlight'})
for index, link in enumerate(links):
    
    #headline
    head = heads[index].text
    headline.append(head)
    
    #htmlbody of teh article
    link = link.attrs['href'] 
    newpage = requests.get(f"https://telex.hu{link}")
    newpage_html = BeautifulSoup(newpage.text, 'html.parser')
    newpage_html.prettify
    
    #title
    title = newpage_html.find('h1')
    titles.append(title.text)
    
    #textbody
    article = newpage_html.find('div', {'class':'article-html-content'}) 
    article_text = article.get_text()
    textbody.append(article_text)

    
# For the other pages
for pagenumber in range(2,cikkszam+1):
    page = requests.get(f"https://telex.hu/legfrissebb/?oldal={pagenumber}")   
    soup = BeautifulSoup(page.text, 'html.parser')
    soup.prettify
    
    links = soup.find_all('a',{'class':'list__item__title'})
    heads = soup.find_all('p',{'class':'list__item__lead hasHighlight'})
    for index, link in enumerate(links):

        #headline
        head = heads[index].text
        headline.append(head)

        #htmlbody of teh article
        link = link.attrs['href'] 
        newpage = requests.get(f"https://telex.hu{link}")
        newpage_html = BeautifulSoup(newpage.text, 'html.parser')
        newpage_html.prettify

        #title
        title = newpage_html.find('h1').text
        titles.append(title)

        #textbody
        article = newpage_html.find('div', {'class':'article-html-content'}) 
        article_text = article.get_text()
        textbody.append(article_text)

df_orig = pd.DataFrame({'Title':titles, 'Headline':headline, 'Textbody':textbody})       

#We don't need the Ukranian - Russian livefeed
df_orig.drop([0], inplace=True)
print('kész')

kész


In [11]:
df_orig.to_csv('cikkek_10k.csv', index=False)

# Data Cleaning

In [ ]:
# replacing \n and \t characters


df = df_orig.apply(lambda x: x.replace('\n', " ", regex=True).replace('\t', " ", regex=True))

# dropping articles based on their length in tokens (words)

bad_length_indexes = []

for index, row in df.iterrows():
  if len(row['Textbody'].split(" ")) < 10 or len(row['Textbody'].split(" ")) > 500:
    bad_length_indexes.append(index)

df = df.drop(bad_length_indexes)

df = df.reset_index(drop=True)
df

,Title,Headline,Textbody
0,Jair Bolsonaro akár embert is evett vol...,"„Azt mondták, ha odamegyek, akkor ennem kell b...",Fokozódik a hangulat a brazil elnökválasztás o...
1,Egyedi mezzel üzentek a sérült Gulácsin...,„Térj vissza erősebben” feliratú mezben vonult...,"Hordágyon kellett levinni Gulácsi Pétert, a RB..."
2,Mutatunk néhány cuki képet a palotási t...,,Egy kedves olvasónk osztott meg velünk képeket...
3,Szabályosan kiütötte edzésen a saját cs...,"Kiszivárgott egy felvétel, amelyen látni, hogy...",Óriási balhé van az NBA-bajnok Golden State Wa...
4,Szabotázs miatt nem jártak órákon át a ...,A német közlekedési miniszter és a Deutsche Ba...,Szabotázsakció miatt órákon át szünetelt a von...
...,...,...,...
1369,Gigantikus arany csodaszarvast készít a...,„A magyarok lélekvezető állatát” egy bronz szi...,Elkészült a látványterve annak a Csodaszarvas ...
1370,Élő adásban kapott sztrókot egy híradós...,Elkezdett küszködni a szöveggel. A kollégái eg...,"A stúdióból, élő adásból kellett kórházba szál..."
1371,A vidéki szállodák forgalma visszaesett...,"A vendégéjszakák száma 15, a szállásdíjbevétel...","Kevesebb belföldi, jóval több külföldi turista..."
1372,"Publicus: Nőtt az elégedetlenek aránya,...",Júliushoz képest négy százalékponttal erősödtek.,Annak ellenére is nőtt a Fidesz–KDNP támogatot...


In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 5.1 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3165346 sha256=aacc44fbb6cfd7cd726def902da523417c6c0b8ea4388676853549c7f6029fea
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
# Language detection using FastText: if not Hungarian, we drop it

import fasttext as ft

# fasttext model
# download and upload this manually or use Google Drive since github can only store up to 100 MB:
# https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

try:
  ft_model = ft.load_model("/content/drive/MyDrive/HGT/lid.176.bin")
except:
  raise ValueError("Upload it to Google Drive and mount it or upload it here!")


not_hun_indexes = []
check_manually_indexes = []

for index, row in df.iterrows():
  try:
    lang = ft_model.predict([row['Textbody']])
    if lang[0][0][0] != '__label__hu':
      not_hun_indexes.append(index)
  except:
    # manual checking
    check_manually_indexes.append(index)
    
df = df.drop(not_hun_indexes)

df = df.reset_index(drop=True)
df   



,Title,Headline,Textbody
0,Jair Bolsonaro akár embert is evett vol...,"„Azt mondták, ha odamegyek, akkor ennem kell b...",Fokozódik a hangulat a brazil elnökválasztás o...
1,Egyedi mezzel üzentek a sérült Gulácsin...,„Térj vissza erősebben” feliratú mezben vonult...,"Hordágyon kellett levinni Gulácsi Pétert, a RB..."
2,Mutatunk néhány cuki képet a palotási t...,,Egy kedves olvasónk osztott meg velünk képeket...
3,Szabályosan kiütötte edzésen a saját cs...,"Kiszivárgott egy felvétel, amelyen látni, hogy...",Óriási balhé van az NBA-bajnok Golden State Wa...
4,Szabotázs miatt nem jártak órákon át a ...,A német közlekedési miniszter és a Deutsche Ba...,Szabotázsakció miatt órákon át szünetelt a von...
...,...,...,...
1337,Gigantikus arany csodaszarvast készít a...,„A magyarok lélekvezető állatát” egy bronz szi...,Elkészült a látványterve annak a Csodaszarvas ...
1338,Élő adásban kapott sztrókot egy híradós...,Elkezdett küszködni a szöveggel. A kollégái eg...,"A stúdióból, élő adásból kellett kórházba szál..."
1339,A vidéki szállodák forgalma visszaesett...,"A vendégéjszakák száma 15, a szállásdíjbevétel...","Kevesebb belföldi, jóval több külföldi turista..."
1340,"Publicus: Nőtt az elégedetlenek aránya,...",Júliushoz képest négy százalékponttal erősödtek.,Annak ellenére is nőtt a Fidesz–KDNP támogatot...
